In [7]:
import pandas as pd

from dowhy import CausalModel

import dowhy
import dowhy.datasets
import statsmodels.api as sm

In [3]:
dowhy.__version__

'0.8'

In [10]:
data

{'df':            X0        X1        W0        W1     v0      y
 0    0.285818 -0.345664 -0.449895  1.215770  False   True
 1    1.603875  1.954200  0.607541 -1.304814  False   True
 2    0.849318 -0.308795 -1.406656  0.721937  False  False
 3   -0.241872  1.015836 -0.432109 -0.194618  False  False
 4    0.269501  1.596790 -1.199372  1.236982  False  False
 ..        ...       ...       ...       ...    ...    ...
 995  1.756557  1.544640  1.465123  1.095065   True   True
 996  0.803074  1.228985  0.212749  1.810608   True   True
 997  1.792432  1.346408 -2.244525  0.869539  False  False
 998  0.386898 -0.525876 -0.922149  1.008202  False  False
 999 -0.068963  1.426994 -1.758588 -0.878508  False  False
 
 [1000 rows x 6 columns],
 'treatment_name': ['v0'],
 'outcome_name': 'y',
 'common_causes_names': ['W0', 'W1'],
 'instrument_names': [],
 'effect_modifier_names': ['X0', 'X1'],
 'frontdoor_variables_names': [],
 'dot_graph': 'digraph {v0->y;W0-> v0; W1-> v0;W0-> y; W1-> y;X0-> y; X1

In [24]:
data = dowhy.datasets.linear_dataset(10, num_common_causes=2, num_samples=1000,
                                     num_instruments=0, num_effect_modifiers=0,
                                     num_treatments=1,
                                     treatment_is_binary=True,
                                    outcome_is_binary=True)

df = data['df']

model = CausalModel(df,
                    data["treatment_name"],data["outcome_name"],
                    data["dot_graph"],
                   missing_nodes_as_confounders=True)

identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

causal_estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.generalized_linear_model",
    confidence_intervals=False,
    test_significance=False,
    method_params = {
        'num_null_simulations':10,
        'num_simulations':10,
        'num_quantiles_to_discretize_cont_cols':10,
        'fit_method': "statsmodels",
        'glm_family': sm.families.Binomial(), # logistic regression
#         'need_conditional_estimates':False
    },
   )
print(causal_estimate.estimator.model.summary())
causal_estimate.interpret()

generalized_linear_model
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      996
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -86.565
Date:                Wed, 15 Nov 2023   Deviance:                       173.13
Time:                        11:26:33   Pearson chi2:                     190.
No. Iterations:                    10   Pseudo R-squ. (CS):             0.6580
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0715      

In [25]:
data['ate']